**Train Test and Cross Validation**

In [12]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score


In [13]:
# Load training data
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

# Display the original shape of the dataset
print(f"🔍 AB - Original dataset shape: {df.shape}")

# Drop missing values
df = df.dropna()

# Display the shape after dropping NaNs
print(f"🧹 AB - After dropping NaNs: {df.shape}")

# Features and target variable
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']


🔍 AB - Original dataset shape: (3268, 14)
🧹 AB - After dropping NaNs: (3268, 14)


In [14]:
# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Display the shape after scaling
print(f"📊 AB - Final dataset shape after scaling: {X_scaled.shape}")


📊 AB - Final dataset shape after scaling: (3268, 13)


In [15]:
# AdaBoost Classifier setup
ab = AdaBoostClassifier(random_state=42)

# Hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150, 200],  # Number of base models (trees) in the ensemble
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Impact of each base model
}

# KFold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV setup with F1-score as the scoring parameter
grid_search = GridSearchCV(ab, param_grid, scoring='f1', cv=kf, n_jobs=-1, verbose=1)


In [16]:
# Fit GridSearchCV on the training data
grid_search.fit(X_scaled, y)

# Best model from GridSearchCV
best_ab = grid_search.best_estimator_

# Print the best hyperparameters and the best score
print(f"✅ AB - Best Parameters: {grid_search.best_params_}")
print(f"✅ AB - K-Fold Mean F1-Score with Best Parameters: {grid_search.best_score_:.4f}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
✅ AB - Best Parameters: {'learning_rate': 0.2, 'n_estimators': 200}
✅ AB - K-Fold Mean F1-Score with Best Parameters: 0.7674


In [17]:
# Predictions on training data
y_pred = best_ab.predict(X_scaled)

# Classification report on the training data
print("📌 AB Classification Report on Training Data:")
print(classification_report(y, y_pred))

# Confusion matrix on the training data
print("🟦 AB - Confusion Matrix on Training Data:")
print(confusion_matrix(y, y_pred))


📌 AB Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.82      0.64      0.72      1638
           1       0.70      0.86      0.77      1630

    accuracy                           0.75      3268
   macro avg       0.76      0.75      0.74      3268
weighted avg       0.76      0.75      0.74      3268

🟦 AB - Confusion Matrix on Training Data:
[[1041  597]
 [ 227 1403]]


In [18]:
# Load validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"
df_valid = pd.read_csv(valid_path)

# Drop any rows with missing values in the validation set
df_valid = df_valid.dropna()

# Prepare features and target for validation data
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Scale the validation data using the previously fitted scaler
X_valid_scaled = scaler.transform(X_valid)

# Display the shape of the validation dataset
print(f"🧪 AB - Validation dataset shape: {X_valid.shape}")


🧪 AB - Validation dataset shape: (364, 13)


In [19]:
# Predictions on validation data
y_pred_valid = best_ab.predict(X_valid_scaled)


In [20]:
# Print classification report for validation data
print(f"📌 AB - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Print confusion matrix for validation data
print("🟦 AB - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


📌 AB - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.62      0.71       178
           1       0.71      0.88      0.78       186

    accuracy                           0.75       364
   macro avg       0.77      0.75      0.75       364
weighted avg       0.77      0.75      0.75       364

🟦 AB - Validation Confusion Matrix:
[[111  67]
 [ 23 163]]


In [21]:
# Recalculate Macro average F1-score on validation data
validation_f1_macro = f1_score(y_valid, y_pred_valid, average='macro')
print(f"🏆 Best Model Validation F1-Score: {validation_f1_macro}")


🏆 Best Model Validation F1-Score: 0.7475961538461539
